In [ ]:
import sys 
import matplotlib.pyplot as plt 
import pandas as pd
from pyarrow import parquet 
import numpy as np
sys.path.append('/home/bekah/gPhoton2')

In [ ]:
# checking petal pattern metadata files 
eclipse = 22644

In [ ]:
aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
metadata  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()

In [ ]:
boresight

In [ ]:
plt.scatter(aspect['ra'],aspect['dec'],s=.1)
plt.scatter(boresight['ra0'],boresight['dec0'])

In [ ]:
# for distributiing legs based on boresight rows 

legframes = []
for leg_ix in boresight.index:
    leg = boresight.loc[leg_ix]
    time_pred = aspect["time"] >= leg["time"]
    if leg_ix != len(boresight) - 1:
        time_pred = time_pred & (
            aspect["time"] < boresight.loc[leg_ix + 1]["time"]
            )
    leg_indices = aspect.loc[time_pred].index
    leg_arrays = {
            'ra0': np.full(len(leg_indices), leg['ra0']),
            'dec0': np.full(len(leg_indices), leg['dec0']),
            'leg': np.full(len(leg_indices), leg_ix, dtype=np.uint8)
        }
    legframes.append(pd.DataFrame(leg_arrays))
distributed = pd.concat(legframes).reset_index(drop=True)
result = pd.concat([aspect, distributed], axis=1)

In [ ]:
aspect['flags'].unique()

In [ ]:
aspect

In [ ]:
#basically I think I need a table that shows what eclipses have >0 legs and how many rows they have in the boresight parquet 
#I also need to know the eclipse number and the obstype

In [ ]:
import pyarrow.dataset as ds

metadata_path = "/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet"
boresight_path = "/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet"
output_path = "uneven_legs.parquet"

metadata_ds = ds.dataset(metadata_path, format="parquet")
filtered_metadata_df = metadata_ds.to_table(filter=(ds.field("legs") > 0)).to_pandas()

boresight_ds = ds.dataset(boresight_path, format="parquet")
boresight_df = boresight_ds.to_table(columns=["eclipse"]).to_pandas()

boresight_counts = boresight_df["eclipse"].value_counts()

filtered_metadata_df["boresight_count"] = filtered_metadata_df["eclipse"].map(boresight_counts).fillna(0).astype(int)

filtered_metadata_df.to_parquet(output_path, index=False)

print(f"Result saved to {output_path}")


In [ ]:
legs = parquet.read_table(output_path).to_pandas()

In [ ]:
eclipse = 4748

aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()

aspect["valid_flag"] = aspect["flags"] % 2 == 0

# Initialize the leg ID
current_leg_id = 0
aspect["leg_id"] = None

for i in range(1, len(aspect)):  # start from the second row
    if aspect.loc[i-1, "valid_flag"] == False and aspect.loc[i, "valid_flag"]:
        current_leg_id = (current_leg_id + 1) if current_leg_id is not None else 0
    elif aspect.loc[i, "valid_flag"]:
        aspect.loc[i, "leg_id"] = current_leg_id

leg_counts = aspect.groupby('leg_id').size()
valid_legs = leg_counts[leg_counts >= 50].index

for leg_id in valid_legs:
    first_row = aspect[aspect['leg_id'] == leg_id].iloc[0]  
    
    new_row = boresight.iloc[-1].copy() if not boresight.empty else pd.Series([None] * len(aspect.columns), index=boresight.columns)
    
    new_row['time'] = first_row_time
    
    boresight = pd.concat([boresight, new_row.to_frame().T], ignore_index=True)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].scatter(aspect['ra'], aspect['dec'], c=aspect['flags'],cmap='BrBG',s=5)
axes[0].set_title("RA vs DEC (colored by flag)")
axes[0].set_xlabel("RA")
axes[0].set_ylabel("DEC")
axes[0].grid(True)

axes[1].scatter(aspect['ra'], aspect['dec'], c=aspect['leg_id'], s=5)
axes[1].set_title("RA vs DEC (colored by leg)")
axes[1].set_xlabel("RA")
axes[1].set_ylabel("DEC")
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
boresight

In [ ]:
aspect

In [ ]:
eclipse = 22644

aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()


aspect["valid_flag"] = aspect["flags"] % 2 == 0
aspect["leg_id"] = None
current_leg_id = None
for i in range(1, len(aspect)):
    if not aspect.loc[i-1, "valid_flag"] and aspect.loc[i, "valid_flag"]:
        current_leg_id = (current_leg_id + 1) if current_leg_id is not None else 0
    elif aspect.loc[i, "valid_flag"]:
        aspect.loc[i, "leg_id"] = current_leg_id

leg_counts = aspect.groupby('leg_id').size()
valid_legs = leg_counts[leg_counts >= 50].index

if not boresight.empty:
    first_valid_leg = aspect[aspect['leg_id'] == valid_legs[0]].iloc[0]
    boresight.iloc[0, boresight.columns.get_loc('time')] = first_valid_leg['time']

for leg_id in valid_legs:
    first_row = aspect[aspect['leg_id'] == leg_id].iloc[0]
    first_row_time = first_row['time']
    new_row = boresight.iloc[-1].copy() if not boresight.empty else pd.Series([None] * len(aspect.columns), index=boresight.columns)
    new_row['time'] = first_row_time
    boresight = pd.concat([boresight, new_row.to_frame().T], ignore_index=True)


In [ ]:
boresight

In [ ]:
# for distributiing legs based on boresight rows 

legframes = []
for leg_ix in boresight.index:
    leg = boresight.loc[leg_ix]
    time_pred = aspect["time"] >= leg["time"]
    if leg_ix != len(boresight) - 1:
        time_pred = time_pred & (
            aspect["time"] < boresight.loc[leg_ix + 1]["time"]
            )
    leg_indices = aspect.loc[time_pred].index
    leg_arrays = {
            'ra0': np.full(len(leg_indices), leg['ra0']),
            'dec0': np.full(len(leg_indices), leg['dec0']),
            'leg': np.full(len(leg_indices), leg_ix, dtype=np.uint8)
        }
    legframes.append(pd.DataFrame(leg_arrays))
distributed = pd.concat(legframes).reset_index(drop=True)
result = pd.concat([aspect, distributed], axis=1)

In [ ]:
result.iloc[0]['time']

In [ ]:
boresight.iloc[0]['time']

In [ ]:
plt.scatter(result['ra'],result['dec'],c=result['leg'])

In [ ]:
result.groupby('leg').size()


In [ ]:


def assign_legs_petal_pattern(aspect, boresight):
    aspect["valid_flag"] = aspect["flags"] % 2 == 0
    aspect["leg_id"] = None
    current_leg_id = None
    for i in range(1, len(aspect)):
        if not aspect.loc[i - 1, "valid_flag"] and aspect.loc[i, "valid_flag"]:
            if current_leg_id is None:
                current_leg_id = 0
                aspect.loc[i, "leg_id"] = current_leg_id
            elif current_leg_id is not None:
                current_leg_id += 1
                aspect.loc[i, "leg_id"] = current_leg_id
        elif aspect.loc[i, "valid_flag"]:
            aspect.loc[i, "leg_id"] = current_leg_id

    leg_counts = aspect.groupby('leg_id').size()
    valid_legs = leg_counts[leg_counts >= 50].index

    if not boresight.empty and len(valid_legs)>0:
        first_valid_leg = aspect[aspect['leg_id'] == valid_legs[0]].iloc[0]
        boresight.iloc[0, boresight.columns.get_loc('time')] = first_valid_leg['time']

    for leg_id in valid_legs:
        if leg_id == 0:
            print("skipping leg 0")
        else:
            first_row = aspect[aspect['leg_id'] == leg_id].iloc[0]
            first_row_time = first_row['time']
            new_row = boresight.iloc[-1].copy() if not boresight.empty else pd.Series([None] * len(aspect.columns),
                                                                                      index=boresight.columns)
            new_row['time'] = first_row_time
            boresight = pd.concat([boresight, new_row.to_frame().T], ignore_index=True)
    return aspect, boresight


In [ ]:
eclipse = 22644

aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()


aspect, boresight = assign_legs_petal_pattern(aspect, boresight)

aspect.to_parquet("aspect.parquet", engine="pyarrow", index=False)
boresight.to_parquet("boresight.parquet", engine="pyarrow", index=False)

In [ ]:
def centile_clip(image, centiles=(1, 99.5)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from astropy.io import fits

eclipse = '23532'
fits_files = [f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b02-image-r.fits",
              f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b03-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b04-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b05-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b06-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b07-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b08-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b09-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b10-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b11-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b12-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b13-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b14-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b15-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b16-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b17-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b18-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b19-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b20-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b21-image-r.fits"    ]

def process_fits_image(file):
    with fits.open(file) as hdul:
        image_data = hdul[1].data  
    stretched_image = centile_clip(image_data)
    return stretched_image
frames = []
for file in fits_files:
    stretched_image = process_fits_image(file)
    norm_image = (255 * (stretched_image - stretched_image.min()) / 
                  (stretched_image.max() - stretched_image.min())).astype(np.uint8)
    frames.append(norm_image)

imageio.mimsave("e23532.gif", frames, duration=0.5)  # Adjust duration as needed


In [ ]:
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from astropy.io import fits


eclipse = '29216'
fits_files = [f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b02-image-r.fits",
              f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b03-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b04-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b05-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b06-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b07-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b08-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b09-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b10-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b11-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b12-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b13-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b14-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b15-image-r.fits",
    f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-ffull-b16-image-r.fits"]



# Function to process images
def process_fits_image(file):
    with fits.open(file) as hdul:
        image_data = hdul[1].data  # Extract first image
    
    image_data = np.array(image_data, dtype=np.float64)

    # Handle NaNs & infinities
    image_data[~np.isfinite(image_data)] = np.nanmedian(image_data)

    # Apply centile_clip function
    stretched_image = centile_clip(image_data)

    return stretched_image

# Determine max shape for padding
max_height, max_width = 0, 0
images = []

for file in fits_files:
    img = process_fits_image(file)
    images.append(img)

    if img.shape[0] > max_height:
        max_height = img.shape[0]
    if img.shape[1] > max_width:
        max_width = img.shape[1]

# Function to pad images
def pad_image(image, target_shape):
    pad_y = target_shape[0] - image.shape[0]
    pad_x = target_shape[1] - image.shape[1]

    pad_top = pad_y // 2
    pad_bottom = pad_y - pad_top
    pad_left = pad_x // 2
    pad_right = pad_x - pad_left

    # Use median padding to prevent artifacts
    median_val = np.nanmedian(image)
    padded_image = np.pad(image, ((pad_top, pad_bottom), (pad_left, pad_right)),
                          mode="constant", constant_values=median_val)
    
    return padded_image

# Process images and save them as frames
frames = []
for i, img in enumerate(images):
    padded_image = pad_image(img, (max_height, max_width))

    # Normalize with better contrast handling
    img_min, img_max = np.nanpercentile(padded_image, [1, 99])  # Avoid extreme outliers
    normalized_image = (255 * (padded_image - img_min) / (img_max - img_min))
    normalized_image = np.clip(normalized_image, 0, 255).astype(np.uint8)

    frames.append(normalized_image)

    # Debug: Save each frame to check if they are distinct
    imageio.imwrite(f"debug_frame_{i}.png", normalized_image)

# Save as GIF with longer delay to ensure visibility
imageio.mimsave("output.gif", frames, duration=1, loop=4)  


In [ ]:
eclipse = 10708

aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()

plt.scatter(aspect['ra'],aspect['dec'],c=aspect['flags'])

In [ ]:
uneven = parquet.read_table("uneven_legs.parquet").to_pandas()
uneven

In [ ]:
eclipse_list = uneven[uneven['boresight_count']==1]['eclipse']

In [ ]:
eclipse_list

In [ ]:
aspect_list = []
boresight_list = []

for eclipse in eclipse_list: 
    print(eclipse)

    aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                               filters=[('eclipse','=',eclipse)]).to_pandas()
    boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                               filters=[('eclipse','=',eclipse)]).to_pandas()
    
    aspect, boresight = assign_legs_petal_pattern(aspect, boresight)

    aspect_list.append(aspect)
    boresight_list.append(boresight)

aspect_combined = pd.concat(aspect_list, ignore_index=True)
boresight_combined = pd.concat(boresight_list, ignore_index=True)

aspect_combined.to_parquet('/home/bekah/gPhoton2/gPhoton/aspect/aspect_combined.parquet', index=False)
boresight_combined.to_parquet('/home/bekah/gPhoton2/gPhoton/aspect/boresight_combined.parquet', index=False)

In [ ]:
#bore_combo = parquet.read_table("boresight_combined.parquet").to_pandas()
bore_combo = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight_combined.parquet').to_pandas()
asp_combo = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect_combined.parquet').to_pandas()


In [ ]:
asp_combo

In [ ]:

aspect_combined = pd.read_parquet('/home/bekah/gPhoton2/gPhoton/aspect/aspect_combined.parquet')

import os
plot_dir = '/media/bekah/BekahA/pp_plots'
os.makedirs(plot_dir, exist_ok=True)

for eclipse, group in aspect_combined.groupby('eclipse'):
    plt.figure(figsize=(6, 5))

    leg_counts = group.groupby('leg_id').size()
    valid_legs = leg_counts[leg_counts >= 50].index

    num_valid_legs = len(valid_legs)
    colors = plt.cm.hsv(np.linspace(0, 1, num_valid_legs))  
    color_map = {leg_id: color for leg_id, color in zip(valid_legs, colors)}

    mask_valid = group['leg_id'].isin(valid_legs)
    mask_invalid = ~mask_valid

    for leg_id, sub_group in group[mask_valid].groupby('leg_id'):
        plt.scatter(sub_group['ra'], sub_group['dec'], color=color_map[leg_id], 
                    alpha=0.7, s=1.0, label=f"Leg {leg_id}")

    if mask_invalid.any():
        plt.scatter(group.loc[mask_invalid, 'ra'], group.loc[mask_invalid, 'dec'], 
                    color='gray', alpha=0.7, s=1)

    plt.xlabel('ra')
    plt.ylabel('dec')
    plt.title(f"e{eclipse}")
    plt.legend(fontsize=4, loc='best', frameon=True, markerscale=0.5)

    plot_path = os.path.join(plot_dir, f"eclipse_{eclipse}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()


In [ ]:
 len(unique_leg_ids)

In [ ]:
color_map

In [ ]:
aspect_combined = pd.read_parquet('/home/bekah/gPhoton2/gPhoton/aspect/aspect_combined.parquet')

plot_dir = '/media/bekah/BekahA/pp_plots'
os.makedirs(plot_dir, exist_ok=True)

for eclipse, group in aspect_combined.groupby('eclipse'):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Create two side-by-side plots

    ax1 = axes[0]
    leg_counts = group.groupby('leg_id').size()
    valid_legs = leg_counts[leg_counts >= 40].index

    num_valid_legs = len(valid_legs)
    colors = plt.cm.hsv(np.linspace(0, 1, num_valid_legs))  # Distinct colors
    color_map_leg = {leg_id: color for leg_id, color in zip(valid_legs, colors)}

    mask_valid = group['leg_id'].isin(valid_legs)
    mask_invalid = ~mask_valid

    for leg_id, sub_group in group[mask_valid].groupby('leg_id'):
        ax1.scatter(sub_group['ra'], sub_group['dec'], color=color_map_leg[leg_id], 
                    alpha=0.7, s=1, label=f"Leg {leg_id}")

    if mask_invalid.any():
        ax1.scatter(group.loc[mask_invalid, 'ra'], group.loc[mask_invalid, 'dec'], 
                    color='gray', alpha=0.7, s=1)

    ax1.set_xlabel("ra")
    ax1.set_ylabel("dec")
    ax1.set_title(f"E{eclipse} - leg_id")
    ax1.legend(fontsize=4, loc='best', frameon=True, markerscale=0.5)

    ax2 = axes[1]
    unique_flags = group['flags'].dropna().unique()
    colors_flags = plt.cm.tab20(np.linspace(0, 1, len(unique_flags)))  
    color_map_flags = {flag: color for flag, color in zip(unique_flags, colors_flags)}

    for flag, sub_group in group.groupby('flags'):
        color = color_map_flags.get(flag, 'gray') 
        ax2.scatter(sub_group['ra'], sub_group['dec'], color=color, 
                    alpha=0.7, s=1,label=f"Flag {flag}")

    ax2.set_xlabel("ra")
    ax2.set_ylabel("dec")
    ax2.set_title(f"E{eclipse} - flags")
    ax2.legend(fontsize=4, loc='best', frameon=True, markerscale=0.5)

    plt.tight_layout()
    plot_path = os.path.join(plot_dir, f"eclipse_{eclipse}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()

print(f"Plots saved in {plot_dir}")


In [ ]:
# need to merge new boresight parquet with old boresight parquet 

bore_combo = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight_combined.parquet').to_pandas()


In [ ]:
bore_combo

In [ ]:
bore_og = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet').to_pandas()


In [ ]:
bore_og

In [ ]:
import pandas as pd

eclipse_values = bore_combo["eclipse"].unique()
bore_og_filtered = bore_og[~bore_og["eclipse"].isin(eclipse_values)]
bore_final = pd.concat([bore_og_filtered, bore_combo], ignore_index=True)
bore_final["eclipse"] = bore_final["eclipse"].astype(int)
bore_final = bore_final.sort_values(by="time").reset_index(drop=True)

bore_final.to_parquet("bore_splitPP.parquet", index=False)


In [ ]:
bore_final

In [ ]:
eclipse = 23532

In [ ]:
aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()

In [ ]:
aspect_combined = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect_combined.parquet', filters=[('eclipse','=',eclipse)]).to_pandas()


In [ ]:
e46646 = aspect[aspect['eclipse']==eclipse]

In [ ]:
plt.scatter(e46646['ra'],e46646['dec'],s=1)

In [ ]:
groups = aspect_combined.groupby('leg_id')

In [ ]:
b46646 = bore_final[bore_final['eclipse']==23532]

In [ ]:
b46646

In [ ]:
eclipse = 46646
b46646 = bore_final[bore_final['eclipse']==eclipse]
plt.scatter(b46646.index,b46646['time'])
plt.scatter(b46646.index[0],bore_og[bore_og['eclipse']==eclipse]['time'],marker='*')

In [ ]:
print(bore_og[bore_og['eclipse']==46646]['time'].values)

In [ ]:
eclipse = 29216 
aspect  = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()
boresight = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet',
                           filters=[('eclipse','=',eclipse)]).to_pandas()

In [ ]:
plt.scatter(aspect['ra'],aspect['dec'],s=1)

In [ ]:
boresight

In [ ]:
1681/15

In [ ]:
first_row = aspect.iloc[[0]]
selected_rows = aspect.iloc[100::100, :]
boresight_df = pd.concat([first_row, selected_rows]).reset_index(drop=True)

boresight_df['ra0'] = boresight['ra0'].iloc[0]  
boresight_df['dec0'] = boresight['dec0'].iloc[0] 
boresight_df['roll0'] = boresight['roll0'].iloc[0]  
boresight_df['eclipse'] = boresight['eclipse'].iloc[0]

boresight_df.to_parquet("boresight.parquet", index=False)



In [ ]:
e = parquet.read_table("boresight.parquet").to_pandas()
e